In [1]:
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt

In [2]:
def open_file():
    with open('ddata_min.pickle', 'rb') as f:
        f_data = pickle.load(f)
    return f_data

In [63]:
def coin_select(name='BTC'):
    f_data = open_file()
    coin_name = name
    s_coin_data = f_data[f_data.permno == coin_name].copy()
    s_coin_data = s_coin_data.reset_index()
    
    return s_coin_data

In [33]:
def RSI(period=30, name='BTC'):
    s_coin_data = coin_select(name=name)
    period = period
    U = np.where(s_coin_data.prc.diff(1) > 0, s_coin_data.prc.diff(1), 0)
    D = np.where(s_coin_data.prc.diff(1) < 0, s_coin_data.prc.diff(1)*(-1), 0)
    AU = pd.DataFrame(U, index=s_coin_data.index).rolling(window=period).mean()
    AD = pd.DataFrame(D, index=s_coin_data.index).rolling(window=period).mean()
    RSI = AU / (AD+AU) * 100
    s_coin_data['RSI'] = RSI
    
    return s_coin_data['RSI']

In [37]:
def MACD(n_short=30, n_long=60, n_signal=6, name='BTC'):
    s_coin_data = coin_select(name=name)
    EMA_short = s_coin_data['prc'].ewm(span=n_short, min_periods=n_short-1).mean()
    EMA_long = s_coin_data['prc'].ewm(span=n_long, min_periods=n_long-1).mean()
    
    s_coin_data['MACD'] = EMA_short - EMA_long
    s_coin_data['MACD_signal'] = s_coin_data['MACD'].ewm(span=n_signal, min_periods=n_signal-1).mean()
    s_coin_data['MACD_diff'] = s_coin_data['MACD'] - s_coin_data['MACD_signal']
    
    return s_coin_data['MACD'], s_coin_data['MACD_signal'], s_coin_data['MACD_diff']

In [41]:
def VWAP(window=300, name='BTC'):
    s_coin_data = coin_select(name=name)
    df_tvol = s_coin_data.tvol.rolling(window=window).sum()
    df_prc_mul_tvol = (s_coin_data.prc*s_coin_data.tvol).rolling(window=window).sum()
    s_coin_data['VWAP'] = df_prc_mul_tvol / df_tvol
    
    return s_coin_data['VWAP']

In [52]:
def base_df(name='BTC'):
    s_coin_data = coin_select(name=name)
    s_coin_data = s_coin_data.join(RSI())
    s_coin_data = s_coin_data.join(VWAP())
    s_coin_data = s_coin_data.join(MACD())
    
    return s_coin_data

In [69]:
def rsi_trading_test(initial_asset=100, sell_fee=0.0004, buy_fee=0.0004):
    
    df_result = pd.DataFrame(columns=['Period', 'Final Asset', 'Winning', 'Losing'])
    s_coin_data = coin_select(name='BTC')
    
    for i in range(10, 101):
        period = i
        print('RSI period is {}'.format(period))
        
        s_coin_data['RSI'] = RSI(period=i)

        buy_price = 0
#         total_profit = 0
#         total_loss = 0
        win_count = 0
        lose_count = 0
        sell_signal = False
        asset = initial_asset
        sell_fee = sell_fee
        buy_fee = buy_fee
        position = False

        for i, row in s_coin_data.iterrows():
            date, price, rsi = row['date'], row['prc'], row['RSI']

            if sell_signal:
                # 수익률
                profit_pct = (price - buy_price) / buy_price - sell_fee
#                 s_coin_data.at[i, 'Profit'] = profit_pct * asset
                asset *= (1+profit_pct)

                #초기화
                buy_price = 0
                position = False
                sell_signal = False

                # 이익
                if profit_pct > 0:
                    win_count += 1
#                     total_profit += profit_pct * asset

                # 손실
                else:
                    lose_count += 1
#                     total_loss += profit_pct * asset


    #             print('SELL: {} | Sell price: {:7.2f} | Total asset: {:7.4f}'.format(date, price, asset), '\n')


            # Buy signal
            elif not(buy_price and position) and rsi <= 30:
                asset = asset * (1-buy_fee)
                buy_price = s_coin_data.prc[i]
                position = True

    #             print('BUY: {} | Buy price: {:7.2f} | Total asset: {:7.4f}'.format(date, buy_price, asset))

            # Sell signal
            elif (position and buy_price) and rsi >= 70:
                sell_signal = True

            elif asset <= 0:
                break

        df_result.loc[period] = [period, asset, win_count, lose_count]
        print('Final Asset: {:10.4f} | Winning Trade: {:7} | Losing Trade: {:7}'.
                    format(asset, win_count, lose_count))
    
    return df_result

In [70]:
rsi_trading_test()

RSI period is 10
Final Asset:     0.0013 | Winning Trade:   11065 | Losing Trade:    7813
RSI period is 11
Final Asset:     0.0087 | Winning Trade:    9712 | Losing Trade:    6588
RSI period is 12
Final Asset:     0.0413 | Winning Trade:    8626 | Losing Trade:    5670
RSI period is 13
Final Asset:     0.2008 | Winning Trade:    7693 | Losing Trade:    4892
RSI period is 14
Final Asset:     0.4479 | Winning Trade:    6953 | Losing Trade:    4167
RSI period is 15
Final Asset:     0.8955 | Winning Trade:    6149 | Losing Trade:    3618
RSI period is 16
Final Asset:     1.6592 | Winning Trade:    5486 | Losing Trade:    3197
RSI period is 17
Final Asset:     3.9045 | Winning Trade:    4895 | Losing Trade:    2847
RSI period is 18
Final Asset:     4.6182 | Winning Trade:    4430 | Losing Trade:    2502
RSI period is 19
Final Asset:     6.2468 | Winning Trade:    3955 | Losing Trade:    2214
RSI period is 20
Final Asset:    13.0261 | Winning Trade:    3588 | Losing Trade:    1957
RSI period

,Period,Final Asset,Winning,Losing
10,10.0,0.001257,11065.0,7813.0
11,11.0,0.008653,9712.0,6588.0
12,12.0,0.041329,8626.0,5670.0
13,13.0,0.200819,7693.0,4892.0
14,14.0,0.447908,6953.0,4167.0
...,...,...,...,...
96,96.0,206.779411,27.0,15.0
97,97.0,241.098447,21.0,16.0
98,98.0,210.857676,25.0,12.0
99,99.0,223.280313,22.0,12.0


In [ ]:
import itertools
def macd_trading_test(initial_asset=100, sell_fee=0.0004, buy_fee=0.0004):
    
    n_short = range(30, 300)
    n_long = range(3, 5)
    n_signal = range(4, 10)
    sls = list(itertools.product(n_short, n_long, n_signal))
    
    index = -1
    
    s_coin_data = coin_select(name='BTC')
    df_result = pd.DataFrame(columns=['Feature', 'Final Asset', 'Winning', 'Losing'])
    
    for i in sls:
        index += 1
        n_short = i[0]
        n_long = i[1] * i[0]
        n_signal = i[2]
        print('MACD feature | short: {}, long: {}, signal: {}'.format(n_short, n_long, n_signal))
        
        s_coin_data['MACD'], s_coin_data['MACD_signal'], s_coin_data['MACD_diff'] = MACD(n_short=n_short, n_long=n_long, n_signal=n_signal)

        buy_price = 0
#         total_profit = 0
#         total_loss = 0
        win_count = 0
        lose_count = 0
        sell_signal = False
        asset = initial_asset
        sell_fee = sell_fee
        buy_fee = buy_fee
        position = False

        for i, row in s_coin_data.iterrows():
            date, price, macd_diff = row['date'], row['prc'], row['MACD_diff']

            if sell_signal:
                # 수익률
                profit_pct = (price - buy_price) / buy_price - sell_fee
#                 s_coin_data.at[i, 'Profit'] = profit_pct * asset
                asset *= (1+profit_pct)

                #초기화
                buy_price = 0
                position = False
                sell_signal = False

                # 이익
                if profit_pct > 0:
                    win_count += 1
#                     total_profit += profit_pct * asset

                # 손실
                else:
                    lose_count += 1
#                     total_loss += profit_pct * asset


    #             print('SELL: {} | Sell price: {:7.2f} | Total asset: {:7.4f}'.format(date, price, asset), '\n')


            # Buy signal
            elif not(buy_price and position) and macd_diff > 0:
                asset = asset * (1-buy_fee)
                buy_price = s_coin_data.prc[i]
                position = True

    #             print('BUY: {} | Buy price: {:7.2f} | Total asset: {:7.4f}'.format(date, buy_price, asset))

            # Sell signal
            elif (position and buy_price) and macd_diff <= 0:
                sell_signal = True

            elif asset <= 0:
                break

        df_result.loc[index] = [i, asset, win_count, lose_count]
        print('Final Asset: {:10.4f} | Winning Trade: {:7} | Losing Trade: {:7}'.
                    format(asset, win_count, lose_count))
    
    return df_result

In [77]:
def vwap_trading_test(initial_asset=100, sell_fee=0.0004, buy_fee=0.0004):
    
    df_result = pd.DataFrame(columns=['Window', 'Final Asset', 'Winning', 'Losing'])
    s_coin_data = coin_select(name='BTC')
    
    
    for i in range(60, 600):
        window = i
        print('VWAP Window is {}'.format(window))
        
        s_coin_data['VWAP'] = VWAP(window=window)

        buy_price = 0
#         total_profit = 0
#         total_loss = 0
        win_count = 0
        lose_count = 0
        sell_signal = False
        asset = initial_asset
        sell_fee = sell_fee
        buy_fee = buy_fee
        position = False

        for i, row in s_coin_data.iterrows():
            date, price, vwap = row['date'], row['prc'], row['VWAP']

            if sell_signal:
                # 수익률
                profit_pct = (price - buy_price) / buy_price - sell_fee
#                 s_coin_data.at[i, 'Profit'] = profit_pct * asset
                asset *= (1+profit_pct)

                #초기화
                buy_price = 0
                position = False
                sell_signal = False

                # 이익
                if profit_pct > 0:
                    win_count += 1
#                     total_profit += profit_pct * asset

                # 손실
                else:
                    lose_count += 1
#                     total_loss += profit_pct * asset


    #             print('SELL: {} | Sell price: {:7.2f} | Total asset: {:7.4f}'.format(date, price, asset), '\n')


            # Buy signal
            elif not(buy_price and position) and (price/vwap > 1):
                asset = asset * (1-buy_fee)
                buy_price = s_coin_data.prc[i]
                position = True

    #             print('BUY: {} | Buy price: {:7.2f} | Total asset: {:7.4f}'.format(date, buy_price, asset))

            # Sell signal
            elif (position and buy_price) and (price/vwap <= 1):
                sell_signal = True

            elif asset <= 0:
                break

        df_result.loc[window] = [window, asset, win_count, lose_count]
        print('Final Asset: {:10.4f} | Winning Trade: {:7} | Losing Trade: {:7}'.
                    format(asset, win_count, lose_count))
    
    return df_result

In [78]:
vwap_result = vwap_trading_test()

VWAP Window is 60
Final Asset:     0.0000 | Winning Trade:    6968 | Losing Trade:   28583
VWAP Window is 61
Final Asset:     0.0000 | Winning Trade:    6889 | Losing Trade:   28441
VWAP Window is 62


KeyboardInterrupt: 